In [1]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
# These top lines are critical for import from another folder
sys.path.insert(1, '/net/capricorn/home/xing/huijing/Segmentation/scripts/vimentin_DIC_segmentation_pipeline/hj_modify_pipe')

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from os import listdir
import cv2
from skimage.io import imread
import numpy as np
from resnet50_nc import res_model
import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.filters import gaussian
from unsharp_mask import unsharp_mask
from matplotlib import pyplot as plt
from skimage import morphology
from skimage.measure import label,regionprops
from scipy import ndimage
from sklearn.utils import class_weight
import tensorflow as tf
from skimage.exposure import equalize_adapthist,rescale_intensity
from cnn_prep_data import prep_icnn_seg_train_data
from image_random_transform import random_rotation,random_zoom,flip_axis

ModuleNotFoundError: No module named 'resnet50_nc'

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 243843632706355735
]


In [3]:
train_path='/home/zoro/Desktop/HK2_train_data/HK2_us_os_icnn_train/train'
weight_file='HK2_icnn_seg.hdf5'

In [4]:
obj_h=128
obj_w=128
input_shape=(obj_h,obj_w,1)
nb_class=3
model=res_model(input_shape,nb_class)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 64, 64)   3200        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 64, 64, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 64, 64, 64)   0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [5]:
data_gen = ImageDataGenerator(zoom_range=(1,1.25),
    rotation_range=180,
    horizontal_flip=True,
    vertical_flip=True)

In [6]:
train_data,train_label=prep_icnn_seg_train_data(train_path,obj_h,obj_w)
print(train_data.shape,train_label.shape)

/home/zoro/anaconda2/lib/python2.7/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))


((36, 128, 128, 1), (36, 3))


In [20]:
# x=train_data[0,:,:,:]
# plt.imshow(x[:,:,0])
# plt.show()
# rx=random_rotation(x,rg=180)
# plt.imshow(rx[:,:,0])
# plt.show()
# zx=random_zoom(x,(0.75,1.25))
# plt.imshow(zx[:,:,0])
# plt.show()
# fx=flip_axis(x,0)
# plt.imshow(fx[:,:,0])
# plt.show()

In [90]:
ylabel=np.argmax(train_label,axis=1)
label_weight=class_weight.compute_class_weight('balanced', np.unique(ylabel), ylabel.flatten())
print(label_weight)

[1.14674441 0.60887513 2.05933682]


In [45]:
def class_weighted_crossentropy(class_weight):  
    def loss(y_true, y_pred):
        y_pred  = tf.clip_by_value(y_pred, 10e-8, 1.-10e-8)
        return -tf.reduce_sum(y_true * class_weight * tf.log(y_pred))
    return loss

In [46]:
model.compile(loss=class_weighted_crossentropy(label_weight), optimizer='adam', metrics=["accuracy"])

In [8]:
batch_size = 50
nb_epoch=500
history =model.fit_generator(data_gen.flow(train_data,train_label,batch_size=batch_size),
                                     steps_per_epoch=len(train_data)/batch_size, epochs=nb_epoch, verbose=1)
model.save_weights(weight_file)